<a href="https://colab.research.google.com/github/crisprking/miniprotein_genai/blob/main/notebooks/01_backbone_RFdiffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# 01 · Backbone generation with RFdiffusion
# One‑click Colab notebook: installs RFdiffusion, generates de novo backbones,
# and lets you preview / download the PDBs.


In [15]:
# 🔧 Environment setup  (≈3 min on first run)
%%bash
set -e
python -m pip install --quiet --upgrade pip
python -m pip install --quiet py3Dmol omegaconf hydra-core jedi icecream \
    git+https://github.com/NVIDIA/dllogger#egg=dllogger
python -m pip install --quiet --no-dependencies \
    dgl==2.0.0 -f https://data.dgl.ai/wheels/cu121/repo.html
python -m pip install --quiet --no-dependencies e3nn==0.5.5 opt_einsum_fx

if [ ! -d RFdiffusion ]; then
  git clone --depth 1 https://github.com/sokrypton/RFdiffusion.git
fi

python -m pip install --quiet ./RFdiffusion/env/SE3Transformer
python -m pip install --quiet git+https://github.com/sokrypton/ColabDesign.git
echo "Environment ready"


Environment ready


  DEPRECATION: Building 'se3-transformer' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'se3-transformer'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [18]:
!pip install --quiet pyrsistent scipy


In [19]:
import sys, pathlib
sys.path.append("/content/RFdiffusion")
from RFdiffusion.inference import utils as rfu
print("RFdiffusion import ✅")



RFdiffusion import ✅


In [ ]:
# ⚙️ Design parameters
name        = "design1"   #@param {type:"string"}
length      = 100         #@param {type:"integer"}
iterations  = 50          #@param ["25","50","100","150","200"] {type:"raw"}
num_designs = 1           #@param {type:"integer"}
print(f"Will create {num_designs} backbone(s) of length {length}")


In [24]:
# 🚀 Generate backbone(s)  — final working version
import subprocess, pathlib, urllib.request, shutil, os

# ------------------------------------------------------------------
# 0.  Parameters (use existing globals if they exist)
name        = globals().get("name", "design1")
length      = globals().get("length", 100)
iterations  = globals().get("iterations", 50)
num_designs = globals().get("num_designs", 1)
# ------------------------------------------------------------------

# 1.  Download model files and schedules once
urls = {
    "Base_ckpt.pt":      "https://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt",
    "Complex_base_ckpt.pt": "https://files.ipd.uw.edu/pub/RFdiffusion/e29311f6f1bf1af907f9ef9f44b8328b/Complex_base_ckpt.pt",
    "schedules.zip":     "https://files.ipd.uw.edu/krypton/schedules.zip",
}
for fname, url in urls.items():
    if not pathlib.Path(fname).exists():
        print("⬇️", fname)
        urllib.request.urlretrieve(url, fname)
        if fname.endswith(".zip"):
            shutil.unpack_archive(fname, ".")
            pathlib.Path(fname).unlink()
model_dir = pathlib.Path("RFdiffusion/models"); model_dir.mkdir(parents=True, exist_ok=True)
for ck in ("Base_ckpt.pt", "Complex_base_ckpt.pt"):
    src, dst = pathlib.Path(ck), model_dir / ck
    if src.exists() and not dst.exists():
        shutil.move(str(src), dst)

# 2.  Build command  (note the **quoted** contig)
contig_arg = f'contigmap.contigs=["{length}"]'        # ← key fix
out_prefix = f"outputs/{name}"
os.makedirs("outputs", exist_ok=True)

cmd = [
    "python", "RFdiffusion/run_inference.py",
    contig_arg,
    f"inference.output_prefix={out_prefix}",
    f"inference.num_designs={num_designs}",
    f"diffuser.T={iterations}",
    "inference.dump_pdb=True",
    "inference.dump_pdb_path=/dev/shm",
]

print("• Running\n", " ".join(cmd), "\n")

# 3.  Stream RFdiffusion log live
with subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) as p:
    for line in p.stdout:
        print(line, end="")
    p.wait()
    rc = p.returncode

if rc != 0:
    raise RuntimeError(f"RFdiffusion failed (exit {rc}) — scroll up for full log")
print("✅  Backbones saved under outputs/")




⬇️ schedules.zip
• Running
 python RFdiffusion/run_inference.py contigmap.contigs=["100"] inference.output_prefix=outputs/design1 inference.num_designs=1 diffuser.T=50 inference.dump_pdb=True inference.dump_pdb_path=/dev/shm 

/content/RFdiffusion/Track_module.py:241: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
[2025-07-26 01:54:30,527][inference.model_runners][INFO] - Reading checkpoint from /content/RFdiffusion/inference/../models/Base_ckpt.pt
This is inf_conf.ckpt_path
/content/RFdiffusion/inference/../models/Base_ckpt.pt
Assembling -model, -diffuser and -preprocess configs from checkpoint
USING MODEL CONFIG: self._conf[model][n_extra_block] = 4
USING MODEL CONFIG: self._conf[model][n_main_block] = 32
USING MODEL CONFIG: self._conf[model][n_ref_block] = 4
USING MODEL CONFIG: self._conf[model][d_msa] = 256
USING MODEL CONFIG: self._conf[model][d_msa_full] = 64
USIN

In [25]:
# 👀 Preview first backbone in 3‑D
import glob, py3Dmol, IPython, pathlib, json
pdb_files = glob.glob(f"outputs/{name}_0.pdb")
if pdb_files:
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
    view.addModel(open(pdb_files[0]).read(),'pdb')
    view.setStyle({'cartoon':{'color':'spectrum'}}); view.zoomTo(); view.show()
else:
    print("No PDB found.")


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [27]:
# 📦  Package and download results
import zipfile, glob, os, google.colab

pdb_pattern = f"outputs/{name}_*.pdb"
pdb_files   = glob.glob(pdb_pattern)

if not pdb_files:
    raise FileNotFoundError(f"No PDB files matching {pdb_pattern}")

zip_path = f"{name}.result.zip"
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
    for f in pdb_files:
        z.write(f, arcname=os.path.basename(f))

print(f"Zipped {len(pdb_files)} file(s) → {zip_path}")
google.colab.files.download(zip_path)
print("📥  If the popup is blocked, use the left‑side file browser to download manually.")



Zipped 1 file(s) → design1.result.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥  If the popup is blocked, use the left‑side file browser to download manually.


In [ ]:
## Usage notes
* **length** = exact backbone length (integer).
* Use **iterations** = 100–150 for harder cases; ≥150 slows down.
* PDBs are dumped to `/content/outputs/…`.
* For oligomeric or motif‑scaffold tasks, switch to Notebook 02 / 03.
